In [1]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline  # Use the imblearn pipeline

from sklearn.utils.validation import column_or_1d
from sklearn.ensemble import RandomForestClassifier

In [2]:
%run ./ml_model_init.ipynb

C:\Users\Ilyas\AppData\Local\Temp\ipykernel_17128\1650975930.py:7: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  data = pd.concat([data_22_23, data_23_24, data_24_25_])


In [3]:
data_tar_preds = data_tar.copy()
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,npg_5,npxG_5,xGChain_5,xGBuildup_5,xP_5,selected_5,pts_bps,whh,whd,wha
19,0.0,0.0,0.0,0.0,0.0,14.0,0.00,0.00,0.00,4.46,...,0.0,0.00,0.44,0.44,4.1,440897.0,1.0,0.351,0.263,0.386
39,0.0,0.0,0.0,0.0,1.9,388.0,0.02,0.04,0.02,1.91,...,0.0,0.41,1.38,0.74,23.9,3414251.0,0.0,0.717,0.164,0.119
57,0.0,0.0,9.0,0.0,13.8,217.0,0.03,0.09,0.06,1.84,...,0.0,0.21,0.58,0.33,9.3,79060.0,1.0,0.543,0.250,0.207
87,0.0,0.0,4.0,0.0,12.0,453.0,0.00,0.00,0.00,2.94,...,0.0,1.18,3.23,0.69,12.0,552787.0,1.0,0.351,0.263,0.386
99,0.0,0.0,2.0,0.0,0.0,463.0,0.00,0.00,0.00,0.96,...,0.0,0.00,0.02,0.00,-0.5,5643.0,1.0,0.351,0.263,0.386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6292,0.0,0.0,-1.0,0.0,14.3,135.0,0.04,0.15,0.11,0.29,...,1.0,0.28,0.76,0.48,9.8,226849.0,0.0,0.225,0.249,0.526
6310,0.0,0.0,19.0,0.0,18.2,600.0,0.01,0.05,0.04,0.88,...,0.0,0.73,1.39,0.61,8.6,18169.0,2.0,0.225,0.249,0.526
6326,0.0,0.0,2.0,0.0,0.7,111.0,0.02,0.10,0.08,0.00,...,0.0,0.27,0.97,0.66,4.6,1471073.0,1.0,0.351,0.263,0.386
6350,1.0,2.0,37.0,1.0,15.9,110.0,0.11,1.49,1.38,0.29,...,1.0,0.70,0.91,0.07,18.1,5515000.0,9.0,0.351,0.263,0.386


## Functions


In [4]:
# For the linear model
def Linear_regression(features_train, features_pred, target_train):
    # bool_cols = features_train.drop(columns=['was_home']).columns.tolist()

    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', LinearRegression())
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    model.fit(features_train, target_train)

    return model.predict(features_pred)

# Decision Tree Model
def DecisionTreeRegression(features_train, features_pred, target_train):
    bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    categorical_cols = ['was_home']

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', DecisionTreeRegressor())
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    model.fit(features_train, target_train)

    return model.predict(features_pred)

# RandomForestRegressor
def RandomForestRegression(features_train, features_pred, target_train, hyperparameters):
    model = TransformedTargetRegressor(RandomForestRegressor(
        n_estimators=hyperparameters['n_estimators'],  max_depth=hyperparameters['max_depth'], criterion=hyperparameters['criterion'], random_state=18), transformer=StandardScaler())
    model.fit(features_train, target_train)

    pred_pred = model.predict(features_pred)

def XGBoostRegression(features_train, features_pred, target_train, hyperparameters):
    regressor = xgb(learning_rate=hyperparameters["learning_rate"],
                    n_estimators=hyperparameters["n_estimators"],
                    max_depth=hyperparameters["max_depth"],
                    eval_metric='rmsle')

    model = TransformedTargetRegressor(regressor, transformer=StandardScaler())


    model.fit(features_train, target_train)

    return model.predict(features_pred)


def Random_Forest_Classifier(features_train, features_pred, target_train, hy_params):
    encoder = LabelEncoder()
    cs_train_ = encoder.fit_transform(target_train)

    # bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    # categorical_cols = ['was_home']
    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False)),
        # ('to_dense', ToDense())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', BorderlineSMOTE(sampling_strategy='auto', random_state=42)),  # Apply SMOTE to the data
        ('classifier', RandomForestClassifier(bootstrap = hy_params['bootstrap'], min_samples_split = hy_params['min_samples_split'], n_estimators =hy_params['n_estimators']  , class_weight='balanced', random_state=42))  # Random Forest Classifier
    ])

    pipeline.fit(features_train, cs_train_)

    # Make predictions
    return pipeline.predict(features_pred)

In [5]:
len(data[data['position'] == 'GK']['element'].unique()) + len(data[data['position'] == 'DEF']['element'].unique()) + len(data[data['position'] == 'MID']['element'].unique()) + len(data[data['position'] == 'FWD']['element'].unique())

1216

In [6]:
len(data[data['position'] == 'GK']['element'].unique()), len(data[data['position'] == 'DEF']['element'].unique()), len(data[data['position'] == 'MID']['element'].unique()), len(data[data['position'] == 'FWD']['element'].unique())

(109, 401, 532, 174)

In [7]:
len(data[(data['position'] != 'GK')]['element'].unique())

754

## Goals Predictor


In [8]:

# (data['minutes_5'] >= 300) &
xg_data= data[(data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',  'ict_index_5',
                                                                     'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5', 'was_home', 'xG', 'xG_3',
                                                                      'xG_5', 'expected_goals_3', 'expected_goals_5', 'goals_scored_3', 'whh', 'whd', 'wha']]

# (data_tar['minutes_5'] >= 300) &
xg_data_tar = data_tar[ (data_tar['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',
                                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3',
                                                                                        'threat_5', 'was_home', 'xG', 'xG_3', 'xG_5', 'expected_goals_3', 'expected_goals_5',
                                                                                        'goals_scored_3','whh', 'whd', 'wha']]

In [9]:
xg_data_tar

,position,minutes_3,minutes_5,team_h_difficulty,team_a_difficulty,ict_index_3,ict_index_5,influence_3,influence_5,creativity_3,...,was_home,xG,xG_3,xG_5,expected_goals_3,expected_goals_5,goals_scored_3,whh,whd,wha
39,DEF,270.0,450.0,2.0,4.0,12.8,21.2,56.2,89.2,57.3,...,False,0.036995,0.41,0.41,0.28,0.28,0.0,0.717,0.164,0.119
57,MID,270.0,360.0,2.0,3.0,8.0,10.2,19.2,33.4,42.5,...,False,0.016875,0.09,0.21,0.09,0.18,0.0,0.543,0.250,0.207
87,FWD,226.0,397.0,2.0,2.0,9.4,14.7,24.8,38.4,31.5,...,True,0.000000,0.33,1.18,0.30,0.84,0.0,0.351,0.263,0.386
99,MID,7.0,7.0,2.0,2.0,2.2,2.2,7.0,7.0,14.8,...,True,0.000000,0.00,0.00,0.00,0.00,0.0,0.351,0.263,0.386
122,DEF,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,...,True,0.011698,0.00,0.00,0.00,0.00,0.0,0.225,0.249,0.526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6292,MID,77.0,168.0,4.0,3.0,3.3,10.9,5.4,44.6,3.4,...,True,0.230644,0.04,0.28,0.05,0.50,0.0,0.225,0.249,0.526
6310,MID,125.0,266.0,4.0,3.0,9.8,18.2,29.6,51.2,27.1,...,True,0.046156,0.60,0.73,0.41,0.59,0.0,0.225,0.249,0.526
6326,MID,108.0,272.0,2.0,2.0,4.5,9.5,25.6,58.6,15.5,...,False,0.111273,0.00,0.27,0.00,0.09,0.0,0.351,0.263,0.386
6350,FWD,254.0,415.0,2.0,2.0,7.0,19.1,6.2,51.2,25.1,...,False,2.399153,0.49,0.70,0.55,0.71,0.0,0.351,0.263,0.386


In [10]:

xg = xg_data[['xG']]
feats = xg_data.drop('xG', axis=1)

xg_tar = xg_data_tar[['xG']].copy()
feats_tar = xg_data_tar.drop('xG', axis=1)

In [11]:
xg_pred = Linear_regression(feats, feats_tar, xg)
xg_tar['xg_pred'] = xg_pred
# xg_tar = xg_tar.drop('xG', axis=1)
# comb_data = xg_data_tar.join(xg_tar)
# comb_data

def add_preds(row):
    if row.name in list(xg_tar.index):
        # print(row.name, xg_tar.loc[row.name, 'xg_pred'])
        row['xg_pred'] = xg_tar.loc[row.name, 'xg_pred']
    else:
        row['xg_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,npxG_5,xGChain_5,xGBuildup_5,xP_5,selected_5,pts_bps,whh,whd,wha,xg_pred
19,0.0,0.0,0.0,0.0,0.0,14.0,0.00,0.00,0.00,4.46,...,0.00,0.44,0.44,4.1,440897.0,1.0,0.351,0.263,0.386,0.000000
39,0.0,0.0,0.0,0.0,1.9,388.0,0.02,0.04,0.02,1.91,...,0.41,1.38,0.74,23.9,3414251.0,0.0,0.717,0.164,0.119,0.060483
57,0.0,0.0,9.0,0.0,13.8,217.0,0.03,0.09,0.06,1.84,...,0.21,0.58,0.33,9.3,79060.0,1.0,0.543,0.250,0.207,0.095729
87,0.0,0.0,4.0,0.0,12.0,453.0,0.00,0.00,0.00,2.94,...,1.18,3.23,0.69,12.0,552787.0,1.0,0.351,0.263,0.386,0.265463
99,0.0,0.0,2.0,0.0,0.0,463.0,0.00,0.00,0.00,0.96,...,0.00,0.02,0.00,-0.5,5643.0,1.0,0.351,0.263,0.386,0.062366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6292,0.0,0.0,-1.0,0.0,14.3,135.0,0.04,0.15,0.11,0.29,...,0.28,0.76,0.48,9.8,226849.0,0.0,0.225,0.249,0.526,0.110177
6310,0.0,0.0,19.0,0.0,18.2,600.0,0.01,0.05,0.04,0.88,...,0.73,1.39,0.61,8.6,18169.0,2.0,0.225,0.249,0.526,0.126854
6326,0.0,0.0,2.0,0.0,0.7,111.0,0.02,0.10,0.08,0.00,...,0.27,0.97,0.66,4.6,1471073.0,1.0,0.351,0.263,0.386,0.055883
6350,1.0,2.0,37.0,1.0,15.9,110.0,0.11,1.49,1.38,0.29,...,0.70,0.91,0.07,18.1,5515000.0,9.0,0.351,0.263,0.386,0.296086


## Assists Predictor


In [12]:

xa_data= data[ (data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'was_home', 'expected_assists', 'xA_3', 'xA_5',  'expected_assists_3', 'expected_assists_5', 'whh', 'whd', 'wha']]

xa_data_tar = data_tar[(data_tar['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5','was_home', 'expected_assists', 'xA_3', 'xA_5',  'expected_assists_3',
                    'expected_assists_5', 'whh', 'whd', 'wha']]



In [13]:

xa = xa_data[['expected_assists']]
feats = xa_data.drop('expected_assists', axis=1)

xa_tar = xa_data_tar[['expected_assists']].copy()
feats_tar = xa_data_tar.drop('expected_assists', axis=1)

In [14]:
xa_pred = Linear_regression(feats, feats_tar, xa)
xa_tar['xa_pred'] = xa_pred
# xa_tar = xa_tar.drop('expected_assists', axis=1)
# comb_data = comb_data.join(xa_tar, rsuffix='x')
# comb_data

def add_preds(row):
    if row.name in list(xa_tar.index):
        # print(row.name, xa_tar.loc[row.name, 'xa_pred'])
        row['xa_pred'] = xa_tar.loc[row.name, 'xa_pred']
    else:
        row['xa_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,xGChain_5,xGBuildup_5,xP_5,selected_5,pts_bps,whh,whd,wha,xg_pred,xa_pred
19,0.0,0.0,0.0,0.0,0.0,14.0,0.00,0.00,0.00,4.46,...,0.44,0.44,4.1,440897.0,1.0,0.351,0.263,0.386,0.000000,0.000000
39,0.0,0.0,0.0,0.0,1.9,388.0,0.02,0.04,0.02,1.91,...,1.38,0.74,23.9,3414251.0,0.0,0.717,0.164,0.119,0.060483,0.103306
57,0.0,0.0,9.0,0.0,13.8,217.0,0.03,0.09,0.06,1.84,...,0.58,0.33,9.3,79060.0,1.0,0.543,0.250,0.207,0.095729,0.067574
87,0.0,0.0,4.0,0.0,12.0,453.0,0.00,0.00,0.00,2.94,...,3.23,0.69,12.0,552787.0,1.0,0.351,0.263,0.386,0.265463,0.036136
99,0.0,0.0,2.0,0.0,0.0,463.0,0.00,0.00,0.00,0.96,...,0.02,0.00,-0.5,5643.0,1.0,0.351,0.263,0.386,0.062366,0.060773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6292,0.0,0.0,-1.0,0.0,14.3,135.0,0.04,0.15,0.11,0.29,...,0.76,0.48,9.8,226849.0,0.0,0.225,0.249,0.526,0.110177,0.060260
6310,0.0,0.0,19.0,0.0,18.2,600.0,0.01,0.05,0.04,0.88,...,1.39,0.61,8.6,18169.0,2.0,0.225,0.249,0.526,0.126854,0.086651
6326,0.0,0.0,2.0,0.0,0.7,111.0,0.02,0.10,0.08,0.00,...,0.97,0.66,4.6,1471073.0,1.0,0.351,0.263,0.386,0.055883,0.053082
6350,1.0,2.0,37.0,1.0,15.9,110.0,0.11,1.49,1.38,0.29,...,0.91,0.07,18.1,5515000.0,9.0,0.351,0.263,0.386,0.296086,0.045563


## Clean Sheets


In [15]:
cs_data= data[(data['position'] != 'FWD')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'clean_sheets', 'clean_sheets_3', 'clean_sheets_5',
                                                                         'expected_goals_conceded_3', 'expected_goals_conceded_5', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'whh', 'whd', 'wha']]

cs_data_tar = data_tar[ (data_tar['position'] != 'FWD')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'clean_sheets', 'clean_sheets_3', 'clean_sheets_5',
                                                                         'expected_goals_conceded_3', 'expected_goals_conceded_5', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'whh', 'whd', 'wha']]

In [16]:
cs = cs_data[['clean_sheets']]
feats = cs_data.drop('clean_sheets', axis=1)

cs_tar = cs_data_tar[['clean_sheets']].copy()
feats_tar = cs_data_tar.drop('clean_sheets', axis=1)

In [17]:
cs_hy_params = {'bootstrap': False, 'min_samples_split': 2, 'n_estimators': 100}
cs_pred = Random_Forest_Classifier(feats, feats_tar, column_or_1d(cs), cs_hy_params)
cs_tar['cs_pred'] = cs_pred
# cs_tar = cs_tar.drop('clean_sheets', axis=1)
# comb_data = comb_data.join(cs_tar, rsuffix='x')
# comb_data

def add_preds(row):
    if row.name in list(cs_tar.index):
        # print(row.name, cs_tar.loc[row.name, 'cs_pred'])
        row['cs_pred'] = cs_tar.loc[row.name, 'cs_pred']
    else:
        row['cs_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,xGBuildup_5,xP_5,selected_5,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred
19,0.0,0.0,0.0,0.0,0.0,14.0,0.00,0.00,0.00,4.46,...,0.44,4.1,440897.0,1.0,0.351,0.263,0.386,0.000000,0.000000,0
39,0.0,0.0,0.0,0.0,1.9,388.0,0.02,0.04,0.02,1.91,...,0.74,23.9,3414251.0,0.0,0.717,0.164,0.119,0.060483,0.103306,0
57,0.0,0.0,9.0,0.0,13.8,217.0,0.03,0.09,0.06,1.84,...,0.33,9.3,79060.0,1.0,0.543,0.250,0.207,0.095729,0.067574,0
87,0.0,0.0,4.0,0.0,12.0,453.0,0.00,0.00,0.00,2.94,...,0.69,12.0,552787.0,1.0,0.351,0.263,0.386,0.265463,0.036136,0
99,0.0,0.0,2.0,0.0,0.0,463.0,0.00,0.00,0.00,0.96,...,0.00,-0.5,5643.0,1.0,0.351,0.263,0.386,0.062366,0.060773,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6292,0.0,0.0,-1.0,0.0,14.3,135.0,0.04,0.15,0.11,0.29,...,0.48,9.8,226849.0,0.0,0.225,0.249,0.526,0.110177,0.060260,0
6310,0.0,0.0,19.0,0.0,18.2,600.0,0.01,0.05,0.04,0.88,...,0.61,8.6,18169.0,2.0,0.225,0.249,0.526,0.126854,0.086651,0
6326,0.0,0.0,2.0,0.0,0.7,111.0,0.02,0.10,0.08,0.00,...,0.66,4.6,1471073.0,1.0,0.351,0.263,0.386,0.055883,0.053082,0
6350,1.0,2.0,37.0,1.0,15.9,110.0,0.11,1.49,1.38,0.29,...,0.07,18.1,5515000.0,9.0,0.351,0.263,0.386,0.296086,0.045563,0


## Yellow Cards


In [18]:
#
yc_data= data[ (data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'yellow_cards', 'yellow_cards_3', 'yellow_cards_5', 'whh', 'whd', 'wha']]

yc_data_tar = data_tar[ (data_tar['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'yellow_cards', 'yellow_cards_3', 'yellow_cards_5', 'whh', 'whd', 'wha']]


In [19]:
yc = yc_data[['yellow_cards']]
feats = yc_data.drop('yellow_cards', axis=1)

yc_tar = yc_data_tar[['yellow_cards']].copy()
feats_tar = yc_data_tar.drop('yellow_cards', axis=1)

In [20]:
params =  {'bootstrap': False, 'min_samples_split': 10, 'n_estimators': 300}
yc_pred = Random_Forest_Classifier(feats, feats_tar, column_or_1d(yc), params)
yc_tar['yc_pred'] = yc_pred
# yc_tar = yc_tar.drop('yellow_cards', axis=1)
# comb_data = comb_data.merge(yc_tar, left_index=True, right_index=True, how='inner')
# comb_data

def add_preds(row):
    if row.name in list(yc_tar.index):
        # print(row.name, yc_tar.loc[row.name, 'yc_pred'])
        row['yc_pred'] = yc_tar.loc[row.name, 'yc_pred']
    else:
        row['yc_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,xP_5,selected_5,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred,yc_pred
19,0.0,0.0,0.0,0.0,0.0,14.0,0.00,0.00,0.00,4.46,...,4.1,440897.0,1.0,0.351,0.263,0.386,0.000000,0.000000,0,0
39,0.0,0.0,0.0,0.0,1.9,388.0,0.02,0.04,0.02,1.91,...,23.9,3414251.0,0.0,0.717,0.164,0.119,0.060483,0.103306,0,0
57,0.0,0.0,9.0,0.0,13.8,217.0,0.03,0.09,0.06,1.84,...,9.3,79060.0,1.0,0.543,0.250,0.207,0.095729,0.067574,0,0
87,0.0,0.0,4.0,0.0,12.0,453.0,0.00,0.00,0.00,2.94,...,12.0,552787.0,1.0,0.351,0.263,0.386,0.265463,0.036136,0,0
99,0.0,0.0,2.0,0.0,0.0,463.0,0.00,0.00,0.00,0.96,...,-0.5,5643.0,1.0,0.351,0.263,0.386,0.062366,0.060773,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6292,0.0,0.0,-1.0,0.0,14.3,135.0,0.04,0.15,0.11,0.29,...,9.8,226849.0,0.0,0.225,0.249,0.526,0.110177,0.060260,0,0
6310,0.0,0.0,19.0,0.0,18.2,600.0,0.01,0.05,0.04,0.88,...,8.6,18169.0,2.0,0.225,0.249,0.526,0.126854,0.086651,0,0
6326,0.0,0.0,2.0,0.0,0.7,111.0,0.02,0.10,0.08,0.00,...,4.6,1471073.0,1.0,0.351,0.263,0.386,0.055883,0.053082,0,0
6350,1.0,2.0,37.0,1.0,15.9,110.0,0.11,1.49,1.38,0.29,...,18.1,5515000.0,9.0,0.351,0.263,0.386,0.296086,0.045563,0,0


## Saves


In [21]:
sv_data= data[ (data['position'] == 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'clean_sheets_3', 'clean_sheets_5', 'expected_goals_conceded_3', 'expected_goals_conceded_5','saves',
                                                                          'saves_3', 'saves_5', 'whh', 'whd', 'wha']]

sv_data_tar = data_tar[(data_tar['position'] == 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'clean_sheets_3', 'clean_sheets_5', 'expected_goals_conceded_3', 'expected_goals_conceded_5','saves',
                                                                          'saves_3', 'saves_5', 'whh', 'whd', 'wha']]


In [22]:
sv = sv_data[['saves']]
feats = sv_data.drop('saves', axis=1)

sv_tar = sv_data_tar[['saves']].copy()
feats_tar = sv_data_tar.drop('saves', axis=1)

In [23]:
sv_pred = Linear_regression(feats, feats_tar, sv)
sv_tar['sv_pred'] = sv_pred
# sv_tar = sv_tar.drop('saves', axis=1)
# comb_data = comb_data.join(sv_tar, rsuffix='x')
# comb_data

def add_preds(row):
    if row.name in list(sv_tar.index):
        # print(row.name, sv_tar.loc[row.name, 'sv_pred'])
        row['sv_pred'] = sv_tar.loc[row.name, 'sv_pred']
    else:
        row['sv_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,selected_5,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred,yc_pred,sv_pred
19,0.0,0.0,0.0,0.0,0.0,14.0,0.00,0.00,0.00,4.46,...,440897.0,1.0,0.351,0.263,0.386,0.000000,0.000000,0,0,3.378631
39,0.0,0.0,0.0,0.0,1.9,388.0,0.02,0.04,0.02,1.91,...,3414251.0,0.0,0.717,0.164,0.119,0.060483,0.103306,0,0,0.000000
57,0.0,0.0,9.0,0.0,13.8,217.0,0.03,0.09,0.06,1.84,...,79060.0,1.0,0.543,0.250,0.207,0.095729,0.067574,0,0,0.000000
87,0.0,0.0,4.0,0.0,12.0,453.0,0.00,0.00,0.00,2.94,...,552787.0,1.0,0.351,0.263,0.386,0.265463,0.036136,0,0,0.000000
99,0.0,0.0,2.0,0.0,0.0,463.0,0.00,0.00,0.00,0.96,...,5643.0,1.0,0.351,0.263,0.386,0.062366,0.060773,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6292,0.0,0.0,-1.0,0.0,14.3,135.0,0.04,0.15,0.11,0.29,...,226849.0,0.0,0.225,0.249,0.526,0.110177,0.060260,0,0,0.000000
6310,0.0,0.0,19.0,0.0,18.2,600.0,0.01,0.05,0.04,0.88,...,18169.0,2.0,0.225,0.249,0.526,0.126854,0.086651,0,0,0.000000
6326,0.0,0.0,2.0,0.0,0.7,111.0,0.02,0.10,0.08,0.00,...,1471073.0,1.0,0.351,0.263,0.386,0.055883,0.053082,0,0,0.000000
6350,1.0,2.0,37.0,1.0,15.9,110.0,0.11,1.49,1.38,0.29,...,5515000.0,9.0,0.351,0.263,0.386,0.296086,0.045563,0,0,0.000000


In [24]:
data_tar_preds[['sv_pred']].max()

sv_pred    3.791921
dtype: float64

## Total Points


In [25]:
# For playing up to 60 minutes	1
# For playing 60 minutes or more (excluding stoppage time)	2
# For each goal scored by a goalkeeper	10
# For each goal scored by a defender	6
# For each goal scored by a midfielder	5
# For each goal scored by a forward	4
# For each goal assist	3
# For a clean sheet by a goalkeeper or defender	4
# For a clean sheet by a midfielder	1
# For every 3 shot saves by a goalkeeper	1
# For each penalty save	5
# For each penalty miss	-2
# Bonus points for the best players in a match	1-3
# For every 2 goals conceded by a goalkeeper or defender	-1
# For each yellow card	-1
# For each red card	-3
# For each own goal	-2

## GK
## ---> >60  = 2
## ---> cs*4
## ---> ((sv)/3)*1
## ---> gc
## ---> yc*-1


## DEF
## ---> >60  = 2
## ---> xg*6
## ---> xa*3
## ---> cs*4
## ---> gc
## ---> yc*-1


## MID
## ---> >60  = +2
## ---> xg*5
## ---> xa*3
## ---> cs*1
## ---> yc*-1


## FWD
## ---> >60  = 2
## ---> xg*4
## ---> xa*3
## ---> yc*-1

In [26]:
def calc_total_pts(row):
    if(row.position == 'GK'):
                ## ---> >60  = 2
        ## ---> cs*4
        ## ---> ((sv)/3)*1
        ## ---> gc
        ## ---> yc*-1
        row['xPts'] = 2 + row['cs_pred']*4 + ((row['sv_pred'])/3)*1 + row['yc_pred']*(-1)

    elif(row.position == 'DEF'):
        ## ---> >60  = 2
        ## ---> xg*6
        ## ---> xa*3
        ## ---> cs*4
        ## ---> gc
        ## ---> yc*-1
        row['xPts'] = 2 + row['xg_pred']*6 + row['xa_pred']*3 + row['cs_pred']*4 + row['yc_pred']*(-1)
    elif(row.position == 'MID'):
        ## MID
        ## ---> >60  = +2
        ## ---> xg*5
        ## ---> xa*3
        ## ---> cs*1
        ## ---> yc*-1

        row['xPts'] = 2 + row['xg_pred']*5 + row['xa_pred']*3 + row['cs_pred']*1 + row['yc_pred']*(-1)

    elif(row.position == 'FWD'):
        ## ---> >60  = 2
        ## ---> xg*4
        ## ---> xa*3
        ## ---> yc*-1

        row['xPts'] = 2 + row['xg_pred']*4 + row['xa_pred']*3 + row['yc_pred']*(-1)

    return row

data_tar_preds = data_tar_preds.apply(calc_total_pts, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred,yc_pred,sv_pred,xPts
19,0.0,0.0,0.0,0.0,0.0,14.0,0.00,0.00,0.00,4.46,...,1.0,0.351,0.263,0.386,0.000000,0.000000,0,0,3.378631,3.126210
39,0.0,0.0,0.0,0.0,1.9,388.0,0.02,0.04,0.02,1.91,...,0.0,0.717,0.164,0.119,0.060483,0.103306,0,0,0.000000,2.672815
57,0.0,0.0,9.0,0.0,13.8,217.0,0.03,0.09,0.06,1.84,...,1.0,0.543,0.250,0.207,0.095729,0.067574,0,0,0.000000,2.681370
87,0.0,0.0,4.0,0.0,12.0,453.0,0.00,0.00,0.00,2.94,...,1.0,0.351,0.263,0.386,0.265463,0.036136,0,0,0.000000,3.170258
99,0.0,0.0,2.0,0.0,0.0,463.0,0.00,0.00,0.00,0.96,...,1.0,0.351,0.263,0.386,0.062366,0.060773,0,0,0.000000,2.494148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6292,0.0,0.0,-1.0,0.0,14.3,135.0,0.04,0.15,0.11,0.29,...,0.0,0.225,0.249,0.526,0.110177,0.060260,0,0,0.000000,2.731664
6310,0.0,0.0,19.0,0.0,18.2,600.0,0.01,0.05,0.04,0.88,...,2.0,0.225,0.249,0.526,0.126854,0.086651,0,0,0.000000,2.894222
6326,0.0,0.0,2.0,0.0,0.7,111.0,0.02,0.10,0.08,0.00,...,1.0,0.351,0.263,0.386,0.055883,0.053082,0,0,0.000000,2.438661
6350,1.0,2.0,37.0,1.0,15.9,110.0,0.11,1.49,1.38,0.29,...,9.0,0.351,0.263,0.386,0.296086,0.045563,0,0,0.000000,3.321032


## Player Points


In [27]:
data_tar_preds[['element', 'position', 'xPts']]

,element,position,xPts
19,14.0,GK,3.126210
39,388.0,DEF,2.672815
57,217.0,MID,2.681370
87,453.0,FWD,3.170258
99,463.0,MID,2.494148
...,...,...,...
6292,135.0,MID,2.731664
6310,600.0,MID,2.894222
6326,111.0,MID,2.438661
6350,110.0,FWD,3.321032


In [30]:
data_tar_preds.to_csv('./predicted.csv', index=False)